In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!ngrok config add-authtoken 2nFtRvqBPidch3zgxg8tUxM8z0K_5ho3vHqrZM8w6BEcHX6P7
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2enqtf7u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2enqtf7u
  Resolved https://github.com/huggingface/transformers to commit 816f4424964c1a1631e303b663fc3d68f731e923
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9991549 sha256=e32c87e9c68c3e839f3bc80559d235e6cf7083627949edc2dfbb121327a1b972
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ck16_v9/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Anamta98/Qwen2-VL-2B-Instruct-LoRA-FT")


# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Anamta98/Qwen2-VL-2B-Instruct-LoRA-FT",
    torch_dtype=torch.bfloat16,
    #attn_implementation="flash_attention_2",
    device_map="auto",
)


# default processer
#processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
from flask import Flask, request, render_template_string, jsonify
from pyngrok import ngrok
import os
import base64
import torch
from transformers import AutoProcessor, AutoModelForSeq2SeqLM
import pandas as pd
from flask import make_response

# Initialize Flask app
app = Flask(__name__)

# Folder to save images
IMAGE_FOLDER = os.path.join(os.getcwd(), 'images')
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)


# Inference logic: Extract Product Name, Freshness Index image
def extract_freshness_details(image_path):
    # Convert image to base64 and prepare the message format for the model
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Prepare the messages for inference
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """What is the name of this fresh produce shown in the image?
Please provide only the value."""},
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """Analyze the provided image and assign a Freshness Index score based on the following scale:
0-19: Unsellable - Completely spoiled, showing mold, decay, or severe damage.
20-39: Poor - Major quality issues like bruising, wilting, or discoloration.
40-59: Fair - Some deterioration with minor blemishes or wilting.
60-79: Good - Mostly fresh with minimal defects, suitable for sale.
80-100: Excellent - Perfectly fresh, vibrant, and free of defects.
Please provide only the numerical score between 0 and 100 as the output.
"""},
            ],
        },

    ]

    extracted_values = []

    for message in messages:
        text = processor.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info([message])
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        generated_ids = model.generate(**inputs, max_new_tokens=20)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        extracted_values.append(output_text[0] if output_text else "NA")
    return extracted_values

# Inference logic: Extract Product Name, Net Content, MRP, and Expiry Date from image
def extract_product_details_from_image(image_path):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """What is the Product Name along with subname(if present)? If not visible, return NA.
Please provide only the value."""},
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """What is the Net Content? If not visible, return NA.
Please provide only the value."""},
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """What is the MRP? If not visible, return NA.
Please provide only the value."""},
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": """What is the Expiry Date? If not visible, return NA.
Please provide only the value."""},
            ],
        },
    ]

    detailsextracted_values = []

    for message in messages:
        text = processor.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info([message])
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        generated_ids = model.generate(**inputs, max_new_tokens=20)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        detailsoutput_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        detailsextracted_values.append(detailsoutput_text[0].strip() if detailsoutput_text else "NA")

    # Unpack the extracted values
    if len(detailsextracted_values) == 4:
        detailsproduct_name, net_content, mrp, expiry_date = detailsextracted_values
    else:
        detailsproduct_name, net_content, mrp, expiry_date = ("NA", "NA", "NA", "NA")

    return detailsproduct_name, net_content, mrp, expiry_date

DETAILS_HTML_TEMPLATE = """
<!DOCTYPE html>
<html>

<head>
    <title>Flipkart GRiD 6.0</title>
    <link rel="icon" type="image/png" href="https://www.shareicon.net/data/128x128/2016/07/09/118732_flipkart_512x512.png">
    <style>
        /* Styling the top menu bar */
        .top-menu-bar {
            justify-content: space-between;
            position: fixed;
            top: 0;
            left: 0;
            display: flex;
            align-items: center;
            width: 100%;
            height: 60px;
            background-color: #2874f0;
            z-index: 10;
            padding: 0 20px;
            box-sizing: border-box;
            box-shadow: 0 2px 2px rgba(0, 0, 0, 0.2);
        }

        .logo{
            height: 70px;
        }

        /* Styling for the rest of the content */
        body {
            background-color: #f0f0f0;
            font-family: Arial, sans-serif;
            margin: 0;
            padding-left: 0px;
            padding-top: 0%;
        }

        #video,
        #canvas {
            box-shadow: 0 1px 1px 0 rgba(0, 0, 0, .2);
            height: auto;
            width: 100%;
            max-width: 800px;
            margin-top: 75px;
            /* Moves the canvas down */
            margin-left: 100px;
            /* Moves the canvas to the right */
            background-color: transparent;
            border: none;
        }

        #canvas {
            display: none;
        }

        .capture-button {
            cursor:pointer;
            font-family: Arial, Helvetica, sans-serif;
            background-color: #ff9f00;
            font-size: 18px;
            padding: 16px 40px;
            color: #fff;
            border-radius: 2px;
            border-color: transparent;
            min-width: 180px;
            position: absolute; /* Set position to fixed */
            right: 350px; /* Position from the right */
            top: 130px; /* Position from the top */
            z-index: 1;
            align-items: center;
            font-weight: 500;
        }

        .extract-details-button {
            cursor:pointer;
            background-color: #fb641b;
            position: absolute;
            color: white;
            right: 127px;
            top: 130px; /* Adjusted to align with Capture button */
            /* Position below the extract name button */
            z-index: 1;
            padding: 16px 40px;
            font-size: 18px;
            border-radius: 2px;
            border-color: transparent;
            min-width: 180px;
            font-weight: 500;
            overflow: hidden;
        }

        .detailsdownload-csv-button {
            cursor: pointer;
            font-family: Arial, Helvetica, sans-serif;
            background-color: #ff9f00;
            font-size: 14px;
            padding: 10px 10px;
            color: #fff;
            border-radius: 2px;
            border-color: transparent;
            min-width: 140px;
            position: absolute;
            /* Set position to fixed */
            right: 148px;
            /* Position from the right */
            top: 596px;
            /* Position from the top */
            z-index: 1;
            align-items: center;
            font-weight: 200;
        }

        button:hover {
            box-shadow: 0 2px 2px 0 rgba(0, 0, 0, .2);
        }

        .details-box {

            top: 190px; /* Adjust top position as needed */
            right: 105px; /* Adjust right position */
            width: 710px; /* Set a fixed width */
            margin: 20px;
            position: absolute;
            padding: 20px;
            background-color: #ffffff;
            border: 1px solid #ccc;
            border-radius: 2px;
            min-height: 450px;
            max-width: 406px;
            border-color: #e4e7ed;
            box-shadow: 0 1px 1px 0 rgba(0, 0, 0, .2);
            font-size: 16px;
            line-height: 1.5;
            color: #191717;
            word-wrap: break-word;
            box-sizing: border-box;
        }

        #details{
            font-family: Arial, Helvetica, sans-serif;
            font-size: 17px; /* Adjust font size */
            color: #878787; /* Adjust color (Flipkart's blue color) */
             /* Make the text bold */
            margin-bottom: 10px; /* Add spacing below the heading */
            margin-top: 0px;
            font-weight: 500;
        }

        .details-box hr {

            border: 0;
            border-top: 1px solid #ccc; /* Thin grey line */
            margin: 4px 0; /* Spacing around the line */
        }
        .menubar-button-container {
            display: flex;
            gap: 2px; /* 3px gap between buttons */
            justify-content: flex-end; /* Align buttons to the right */
        }
        .freshness-index-button,
        .details-extractor-button {
            padding: 8px 16px;
    background-color: transparent;
    color: #ffffff;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 15px;
    font-weight: 520;
    text-decoration: none;
    font-family: Arial, Helvetica, sans-serif;
        }

        .freshness-index-button:hover,
        .details-extractor-button:hover {
            color: #dddada;
        }

.spinner {
    position: absolute;
    top: 15px;
    right: 17px;
    width: 18px;
    height: 18px;
    border: 4px solid rgba(0, 0, 255, 0.7);
    border-top: 4px solid #ffff;
    border-radius: 50%;
    animation: spin 2s linear infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
    </style>

</head>

<body>
    <!-- Top Menu Bar -->
    <div class="top-menu-bar">
        <img class="logo" src="https://i.ibb.co/DRLGQWY/flipkartlogo.png" alt="Flipkart Logo"></img>
        <div class="menubar-button-container">
<a href="/" id="detailsExtractorNav" class="details-extractor-button">Details Extraction</a>
<a href="/freshnessIndexPage" id="freshnessIndexNav" class="freshness-index-button">Freshness Detection</a>

</div>
    </div>
    <video id="video" autoplay></video>
    <canvas id="canvas"></canvas>
    <br>
    <form id="uploadForm" method="POST" enctype="multipart/form-data">
        <input type="hidden" name="imageData" id="imageData">
        <button type="button" id="snapAndSave" class="capture-button">Capture</button>
    </form>

    <!-- Single Extract Details Button -->
    <button type="button" id="detailsextractProductDetails" class="extract-details-button">Extract Details</button>



    <!-- Download CSV Button -->
    <button type="button" id="detailsdownloadCSV" class="detailsdownload-csv-button">Download Details</button>

    <div id="productDetailsContainer" class="details-box">
        <h4 id="details">DETAILS: </h4>
        <div id="loadingSpinner" class="spinner" style="display: none;"></div>
        <hr>
        <p>Product Name: <span id="detailsproductName"></span></p>
        <p>Net Content: <span id="productNetContent"></span></p>
        <p>MRP: <span id="productMRP"></span></p>
        <p>Expiry Date: <span id="productExpiryDate"></span></p>
    </div>

    <script>
        const loadingSpinner = document.getElementById('loadingSpinner');
        const video = document.getElementById('video');
        const canvas = document.getElementById('canvas');
        const snapAndSaveButton = document.getElementById('snapAndSave');
        const detailsextractProductDetailsButton = document.getElementById('detailsextractProductDetails');
        const detailsdownloadCSVButton = document.getElementById('detailsdownloadCSV'); // Ensure unique ID
        const imageDataInput = document.getElementById('imageData');
        const detailsproductNameDisplay = document.getElementById('detailsproductName');
        const productNetContentDisplay = document.getElementById('productNetContent');
        const productMRPDisplay = document.getElementById('productMRP');
        const productExpiryDateDisplay = document.getElementById('productExpiryDate');

        const constraints = {
            video: true
        };

        navigator.mediaDevices.getUserMedia(constraints).then((stream) => {
            video.srcObject = stream;
        });

        snapAndSaveButton.addEventListener('click', () => {
          loadingSpinner.style.display = 'block';
            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            context.drawImage(video, 0, 0, canvas.width, canvas.height);

            video.style.display = 'none';
            canvas.style.display = 'block';

            const imageData = canvas.toDataURL('image/png');
            imageDataInput.value = imageData;

            document.getElementById('uploadForm').submit();
            loadingSpinner.style.display = 'none';
        });


        detailsextractProductDetailsButton.addEventListener('click', () => {
          loadingSpinner.style.display = 'block';
            fetch('/detailsextract-details', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({}),
            })
                .then(response => response.json())
                .then(data => {
                    if (data.error) {
                        alert(data.error);
                        return;
                    }
                    loadingSpinner.style.display = 'none';
                    // Update the display fields
                    detailsproductNameDisplay.textContent = data.detailsproduct_name;
                    productNetContentDisplay.textContent = data.net_content;
                    productMRPDisplay.textContent = data.mrp;
                    productExpiryDateDisplay.textContent = data.expiry_date;
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('An error occurred while extracting details.');
                    loadingSpinner.style.display = 'none';
                });

        });

        detailsdownloadCSVButton.addEventListener('click', () => {
          loadingSpinner.style.display = 'block';
            fetch('/detailsdownload-csv', {
                method: 'GET',
            })
            .then(response => {
                if (response.status === 404) {
                    return response.json().then(data => { throw new Error(data.error); });
                }
                return response.blob();
            })

            .then(blob => {
                // Create a blob URL and trigger the download
                const url = URL.createObjectURL(blob);
                const link = document.createElement("a");
                link.href = url;
                link.download = "product_details.csv";
                document.body.appendChild(link);
                link.click();
                document.body.removeChild(link);
                loadingSpinner.style.display = 'none';
            })
            .catch(error => {
                console.error('Error:', error);
                alert(error.message);
                loadingSpinner.style.display = 'none';
            });
        });
    </script>
</body>

</html>
"""



# HTML template for the camera interface with two buttons
FRESHNESS_HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flipkart GRiD 6.0</title>
    <link rel="icon" type="image/png" href="https://www.shareicon.net/data/128x128/2016/07/09/118732_flipkart_512x512.png">
    <style>
        /* =======================
           Global Styles
        ======================= */
        body {
            background-color: #f0f0f0;
            font-family: Arial, sans-serif;
            margin: 0;
            padding-top: 60px; /* To prevent overlap with fixed top bar */
        }

        /* =======================
           Top Menu Bar
        ======================= */
        .top-menu-bar {
            justify-content: space-between;
            position: fixed;
            top: 0;
            left: 0;
            display: flex;
            align-items: center;
            width: 100%;
            height: 60px;
            background-color: #2874f0;
            z-index: 10;
            padding: 0 20px;
            box-sizing: border-box;
            box-shadow: 0 2px 2px rgba(0, 0, 0, 0.2);
        }

        .logo{
            height: 70px;
        }

        /* =======================
           Main Container with Flexbox
        ======================= */
        .main-container {
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            padding: 20px;
            gap: 20px;
            max-width: 1200px;
            margin: 0 auto;
        }

        /* =======================
           Camera Section
        ======================= */
        .camera-section {
            flex: 1;
            max-width: 60%;
            position: relative;
        }

        #video,
        #canvas {
            width: 100%;
            max-width: 100%;
            box-shadow: 0 1px 1px rgba(0, 0, 0, 0.2);
            background-color: transparent;
            border: none;
        }

        #canvas {
            display: none;
        }

        /* =======================
           Buttons Styling
        ======================= */
        .button-group {
            display: flex;
            gap: 10px;
            margin-top: 10px;
        }

        .capture-button,
        .extract-details-button,
        .freshnessdownload-csv-button {
            cursor: pointer;
            font-family: Arial, Helvetica, sans-serif;
            color: #fff;
            border: none;
            border-radius: 4px;
            padding: 12px 24px;
            font-size: 16px;
            transition: box-shadow 0.3s ease;
        }

        .capture-button {
            background-color: #ff9f00;
        }

        .extract-details-button {
            background-color: #fb641b;
        }

        .freshnessdownload-csv-button {
            background-color: #ff9f00;
            font-size: 14px;
            padding: 10px 20px;
        }

        .capture-button:hover,
        .extract-details-button:hover,
        .freshnessdownload-csv-button:hover {
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2);
        }

        /* =======================
           Details Box Styling
        ======================= */
        .details-box {
            margin-top: 20px;
            padding: 20px;
            background-color: #ffffff;
            border: 1px solid #e4e7ed;
            border-radius: 4px;
            box-shadow: 0 1px 1px rgba(0, 0, 0, 0.2);
            font-size: 15px;
            color: #191717;
        }

        #details {
            font-size: 16px;
            color: #878787;
            margin-bottom: 10px;
            margin-top: 1px;
            font-weight: 500;
        }

        .details-box hr {
            border: none;
            border-top: 1px solid #ccc;
            margin: 10px 0;
        }

        /* =======================
           Freshness Table Styling
        ======================= */
        .table-container {
            margin-top: 5px;
            flex: 0 0 35%;
        }

        .table-container p {

            color: #343a40;
            margin-bottom: 10px;
        }

        .freshness-table {
            width: 100%;
            border-collapse: collapse;
            background-color: #ffffff;
            border: 1px solid #626262;
            border-radius: 2px;
            box-shadow: 0 1px 1px rgba(0, 0, 0, 0.2);
            overflow: hidden;
        }

        .freshness-table th,
        .freshness-table td {
            padding: 8px 15px;
            text-align: center;
            font-size: 13px;
        }

        .freshness-table thead {
            background-color: #007bff;
            color: #ffffff;
        }

        .freshness-table tbody tr:nth-child(even) {
            background-color: #f2f2f2;
        }

        .freshness-table tbody tr:hover {
            background-color: #e9ecef;
        }

        /* Badge Styles */
        .badge {
            display: inline-block;
            padding: 5px 10px;
            border-radius: 12px;
            color: #ffffff;
            font-weight: bold;
            font-size: 12px;
        }

        .badge-excellent {
            background-color: #28a745; /* Green */
        }

        .badge-good {
            background-color: #17a2b8; /* Teal */
        }

        .badge-fair {
            background-color: #ffc107; /* Yellow */
            color: #343a40;
        }

        .badge-poor {
            background-color: #fd7e14; /* Orange */
        }

        .badge-unsellable {
            background-color: #dc3545; /* Red */
        }
        .menubar-button-container {
            display: flex;
            gap: 2px; /* 3px gap between buttons */
            justify-content: flex-end; /* Align buttons to the right */
        }
        .freshness-index-button,
        .details-extractor-button {
            padding: 8px 16px;
    background-color: transparent;
    color: #ffffff;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 15px;
    font-weight: 520;
    text-decoration: none;
    font-family: Arial, Helvetica, sans-serif;
        }

        .freshness-index-button:hover,
        .details-extractor-button:hover {
            color: #dddada;
        }
    .spinner {
            position: absolute;
            margin-left: 1150px; /* Align to right edge */
            margin-top: 40px; /* Adjust vertically */
            width: 18px;
            height: 18px;
            border: 4px solid rgba(0, 0, 255, 0.7);
            border-top: 4px solid #fff;
            border-radius: 50%;
            animation: spin 2s linear infinite;
            z-index: 9999;
        }

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

    </style>
</head>

<body>
    <!-- Top Menu Bar -->
    <div class="top-menu-bar">
        <img class="logo" src="https://i.ibb.co/DRLGQWY/flipkartlogo.png" alt="Flipkart Logo">

        <div class="menubar-button-container">
<a href="/" id="detailsExtractorNav" class="details-extractor-button">Details Extraction</a>
<a href="/freshnessIndexPage" id="freshnessIndexNav" class="freshness-index-button">Freshness Detection</a>

</div>
    </div>

    <!-- Main Container -->
    <div class="main-container">
        <!-- Camera Section -->
        <div class="camera-section">
        <div id="loadingSpinner" class="spinner" style="display: none;"></div>
            <video id="video" autoplay></video>
            <canvas id="canvas"></canvas>


            <!-- Buttons Group -->
            <div class="button-group">
                <form id="uploadForm" method="POST" enctype="multipart/form-data">
        <input type="hidden" name="imageData" id="imageData">
        <button type="button" id="snapAndSave" class="capture-button">Capture</button>
    </form>
                <button type="button" id="extractProductDetails" class="extract-details-button">Detect Freshness</button>
                <button type="button" id="freshnessdownloadCSV" class="freshnessdownload-csv-button">Download Details</button>
            </div>

            <!-- Details Box -->

        </div>

        <!-- Freshness Table Section -->
        <div class="table-container">
            <div id="productDetailsContainer" class="details-box">
                <h4 id="details">DETAILS: </h4>
                <hr>
                <p>Product Name: <span id="productName"></span></p>
                <p>Freshness Index: <span id="productFreshnessIndex"></span></p>
            </div>
            <br>
            <br>
            <p style="text-align: center";>Freshness Index Scale for Fresh Produce</p>
            <table class="freshness-table">
                <thead>
                    <tr>
                        <th>Freshness Level</th>
                        <th>Score Range</th>
                        <th>Description</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td><span class="badge badge-excellent">Excellent</span></td>
                        <td>80-100</td>
                        <td>Peak freshness, optimal quality</td>
                    </tr>
                    <tr>
                        <td><span class="badge badge-good">Good</span></td>
                        <td>60-79</td>
                        <td>Fresh with minor quality reductions</td>
                    </tr>
                    <tr>
                        <td><span class="badge badge-fair">Fair</span></td>
                        <td>40-59</td>
                        <td>Noticeable quality decline, nearing spoilage</td>
                    </tr>
                    <tr>
                        <td><span class="badge badge-poor">Poor</span></td>
                        <td>20-39</td>
                        <td>Significant spoilage, not ideal for consumption</td>
                    </tr>
                    <tr>
                        <td><span class="badge badge-unsellable">Unsellable</span></td>
                        <td>0-19</td>
                        <td>Completely spoiled, unsuitable for sale or consumption</td>
                    </tr>
                </tbody>
            </table>
        </div>
    </div>

    <!-- JavaScript Section -->
    <script>
        const loadingSpinner = document.getElementById('loadingSpinner');
        const video = document.getElementById('video');
        const canvas = document.getElementById('canvas');
        const snapAndSaveButton = document.getElementById('snapAndSave');
        const extractProductDetailsButton = document.getElementById('extractProductDetails');
        const freshnessdownloadCSVButton = document.getElementById('freshnessdownloadCSV'); // Ensure unique ID
        const imageDataInput = document.getElementById('imageData');
        const productNameDisplay = document.getElementById('productName');
        const productFreshnessIndex = document.getElementById('productFreshnessIndex');

        const constraints = {
            video: true
        };

        // Initialize webcam stream
        navigator.mediaDevices.getUserMedia(constraints).then((stream) => {
            video.srcObject = stream;
        });

        // Capture and send image data to the server via AJAX (Fetch API)
        snapAndSaveButton.addEventListener('click', () => {

            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            context.drawImage(video, 0, 0, canvas.width, canvas.height);

            video.style.display = 'none';
            canvas.style.display = 'block';

            const imageData = canvas.toDataURL('image/png');
            imageDataInput.value = imageData;

            document.getElementById('uploadForm').submit();

        });


        extractProductDetailsButton.addEventListener('click', () => {
          loadingSpinner.style.display = 'block';
            fetch('/extract-details', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({}),
            })
                .then(response => response.json())
                .then(data => {
                    if (data.error) {
                        alert(data.error);
                        return;
                    }
                loadingSpinner.style.display = 'none';

                    // Update the display fields
                    productNameDisplay.textContent = data.product_name;
                    productFreshnessIndex.textContent = data.freshness_index;
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('An error occurred while extracting details.');
                    loadingSpinner.style.display = 'none';
                });
        });

        // Download the extracted details as a CSV file
        freshnessdownloadCSVButton.addEventListener('click', () => {
          loadingSpinner.style.display = 'block';
            fetch('/freshnessdownload-csv', {
                method: 'GET',
            })
                .then(response => {
                    if (response.status === 404) {
                        return response.json().then(data => { throw new Error(data.error); });
                    }
                    return response.blob();
                })
                .then(blob => {
                    // Create a blob URL and trigger the download
                    const url = URL.createObjectURL(blob);
                    const link = document.createElement("a");
                    link.href = url;
                    link.download = "product_details.csv";
                    document.body.appendChild(link);
                    link.click();
                    document.body.removeChild(link);
                     loadingSpinner.style.display = 'none';
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert(error.message);
                     loadingSpinner.style.display = 'none';
                });
        });
    </script>
</body>

</html>

"""

@app.route("/freshness", methods=["GET", "POST"])
def freshnessindex():
    if request.method == "POST":
        image_data = request.form.get("imageData")
        if image_data:
            # Remove the data URL prefix and decode the base64 data
            image_data = image_data.split(",")[1]
            image_data = base64.b64decode(image_data)

            image_path = os.path.join("captured_image.jpg")
            with open(image_path, "wb") as f:
                f.write(image_data)

    return render_template_string(FRESHNESS_HTML_TEMPLATE)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        image_data = request.form.get("imageData")
        if image_data:
            # Remove the data URL prefix and decode the base64 data
            image_data = image_data.split(",")[1]
            image_data = base64.b64decode(image_data)

            image_path = os.path.join("captured_image.jpg")
            with open(image_path, "wb") as f:
                f.write(image_data)

    return render_template_string(DETAILS_HTML_TEMPLATE)

@app.route("/freshnessIndexPage", methods=["GET", "POST"])
def freshnessIndexPagefunction():
    if request.method == "POST":
        image_data = request.form.get("imageData")
        if image_data:
            # Remove the data URL prefix and decode the base64 data
            image_data = image_data.split(",")[1]
            image_data = base64.b64decode(image_data)

            image_path = os.path.join("captured_image.jpg")
            with open(image_path, "wb") as f:
                f.write(image_data)
    return render_template_string(FRESHNESS_HTML_TEMPLATE)

@app.route("/extract-details", methods=["POST"])
def extract_freshness_index():
    image_path = os.path.join("captured_image.jpg")

    if os.path.exists(image_path):
        # Extract Product Name, Freshness Index
        product_name, freshness_index = extract_freshness_details(image_path)

        return jsonify({
            "product_name": product_name,
            "freshness_index": freshness_index

        })
    else:
        return jsonify({"error": "Image not found"}), 404

@app.route("/detailsextract-details", methods=["POST"])
def extract_details():
    image_path = os.path.join("captured_image.jpg")

    if os.path.exists(image_path):
        # Extract Product Name, Net Content, MRP, and Expiry Date
        detailsproduct_name, net_content, mrp, expiry_date = extract_product_details_from_image(image_path)

        return jsonify({
            "detailsproduct_name": detailsproduct_name,
            "net_content": net_content,
            "mrp": mrp,
            "expiry_date": expiry_date
        })
    else:
        return jsonify({"error": "Image not found"}), 404

@app.route("/detailsdownload-csv", methods=["GET"])
def download_csv():
    image_path = os.path.join("captured_image.jpg")

    if os.path.exists(image_path):
        # Extract values
        detailsproduct_name, net_content, mrp, expiry_date = extract_product_details_from_image(image_path)

        # Prepare the data for CSV
        data = {
            "Product Name": [detailsproduct_name],
            "Net Content": [net_content],
            "MRP": [mrp],
            "Expiry Date": [expiry_date]
        }
        df = pd.DataFrame(data)

        # Create a CSV response
        response = make_response(df.to_csv(index=False))
        response.headers["Content-Disposition"] = "attachment; filename=product_details.csv"
        response.headers["Content-Type"] = "text/csv"

        return response
    else:
        return jsonify({"error": "Image not found"}), 404

@app.route("/freshnessdownload-csv", methods=["GET"])
def freshnessdownload_csv():
    image_path = os.path.join("captured_image.jpg")

    if os.path.exists(image_path):
        # Extract values
        product_name, freshness_index = extract_freshness_details(image_path)


        # Prepare the data for CSV
        data = {
            "Product Name": [product_name],
            "Freshness Index": [freshness_index]

        }
        df = pd.DataFrame(data)

        # Create a CSV response
        response = make_response(df.to_csv(index=False))
        response.headers["Content-Disposition"] = "attachment; filename=product_details.csv"
        response.headers["Content-Type"] = "text/csv"

        return response
    else:
        return jsonify({"error": "Image not found"}), 404

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel URL: {public_url}")

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5000)


 * ngrok tunnel URL: NgrokTunnel: "https://7ed1-34-125-239-0.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:26:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:28:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:28:42] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:28:55] "POST /detailsextract-details HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:29:10] "GET /detailsdownload-csv HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:30:16] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:31:22] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:31:32] "POST /detailsextract-details HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:31:59] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Oct/2024 12:32:28] "POST / HT